<a href="https://colab.research.google.com/github/changsin/Kaggle30Days/blob/main/insurance_claims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance Claim Data

## Setup

In [1]:
!git clone https://github.com/changsin/Kaggle30Days

Cloning into 'Kaggle30Days'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 150 (delta 82), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (150/150), 65.31 MiB | 31.13 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [2]:
DATA_ROOT = '/content/Kaggle30Days/data/insurance_claims/'

## Step 1: Import libraries

In [3]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# For training random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Step 2: Load the data

In [4]:
# Load the training data
train = pd.read_csv(DATA_ROOT + "train.csv", index_col=0)
test = pd.read_csv(DATA_ROOT + "test.csv", index_col=0)

# Preview the data
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,B,B,A,A,B,D,A,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,A,A,A,C,B,D,A,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,B,B,A,C,B,D,A,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,A,A,A,C,B,D,A,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


## Step 3: Prepare the data

In [6]:
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

# Step 4: Train

### RandomForest

In [8]:
# Define the model 
model = RandomForestRegressor(random_state=1)

# Train the model (will take about 10 minutes to run)
model.fit(X_train, y_train)
preds_valid = model.predict(X_valid)
print(mean_squared_error(y_valid, preds_valid, squared=False))

KeyboardInterrupt: ignored

### XGBoost

In [9]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model_xgb = XGBRegressor(n_estimators=1000, learning_rate=0.05)
model_xgb.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
preds_xgb = model_xgb.predict(X_valid)
print(mean_squared_error(y_valid, preds_xgb, squared=False))

[11:47:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.7242062379086012


In [13]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

preds_lr = model_lr.predict(X_valid)
print(mean_squared_error(y_valid, preds_lr, squared=False))

0.7415587016098646


In [16]:
from sklearn.linear_model import BayesianRidge

model_b = BayesianRidge()
model_b.fit(X_train, y_train)

preds_b = model_b.predict(X_valid)
print(mean_squared_error(y_valid, preds_b, squared=False))

0.7415560714250127


In [33]:
X_train.columns

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

model = Sequential()
# model.add(Conv1D(24, 2, activation="relu", input_shape=(24, 1)))
# model.add(Flatten())
model.add(Dense(24, activation="relu"))
model.add(Dense(12, activation="relu"))
model.add(Dense(6, activation="relu"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")

model.fit(X_train, y_train, batch_size=24, epochs=50, verbose=0)

model.summary()

In [40]:
preds_nn = model.predict(X_valid)
print(mean_squared_error(y_valid, preds_nn, squared=False))

0.7407928261977715


In [11]:
import numpy as np
from scipy.stats import norm

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors =  np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
          X_c = X[y==c]
          self._mean[idx, :] = X_c.mean(axis=0)
          self._var[idx, :] = X_c.var(axis=0)
          self._priors[idx] = float(X_c.shape[0]) / float(n_samples)
          # print("priors", idx, c, self._priors[idx])

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
            

    # probability density function
    # NB; the original code had minor bugs which I fixed below.
    #   the original code was
    #     numerator = np.exp(- (x-mean)**2 / (2 * var))
    #     denominator = np.sqrt(2 * np.pi * var)
    #
    #   Note that var was not squared.
    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]

        # This is just an implementation of the normal density function.
        # The results are identical with the standard library function
        #
        numerator = np.exp(- (x-mean)**2 / (2 * var**2))
        denominator = np.sqrt(2 * np.pi * var**2)
        class_cps = numerator / denominator
        #
        # class_cps = norm.pdf(x, loc=mean, scale=var)

        # print("class_conditional probabilities", class_cps)
        # print("  norm.pdf", norm.pdf(x, loc=mean, scale=var))

        return class_cps


In [12]:
model_nb = NaiveBayes()
model_nb.fit(X_train, y_train)
preds_nb = model_nb.predict(X_valid)

Streaming output truncated to the last 5000 lines.
priors 219783 9.94865717204716 4.444444444444444e-06
priors 219784 9.94870441321453 4.444444444444444e-06
priors 219785 9.948707258679283 4.444444444444444e-06
priors 219786 9.948781448680384 4.444444444444444e-06
priors 219787 9.948862824777082 4.444444444444444e-06
priors 219788 9.94891677329439 4.444444444444444e-06
priors 219789 9.949020210382539 4.444444444444444e-06
priors 219790 9.949060507650614 4.444444444444444e-06
priors 219791 9.949162955483976 4.444444444444444e-06
priors 219792 9.949191265052331 4.444444444444444e-06
priors 219793 9.94922726313199 4.444444444444444e-06
priors 219794 9.949256729505539 4.444444444444444e-06
priors 219795 9.949268947657789 4.444444444444444e-06
priors 219796 9.949287740308275 4.444444444444444e-06
priors 219797 9.949352232507348 4.444444444444444e-06
priors 219798 9.949399045921938 4.444444444444444e-06
priors 219799 9.94945227394731 4.444444444444444e-06
priors 219800 9.94950911990022 4.444

UFuncTypeError: ignored

In [ ]:
from sklearn import svm

svr = svm.SVR()
svr.fit(X_train, y_train)

preds_svr = svr.predict(X_valid)
print(mean_squared_error(y_valid, preds_svr, squared=False))

In [ ]:

def train_regressor(n_estimators=100, learning_rate=0.05):
  model_xgb = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
  model_xgb.fit(X_train, y_train, 
              early_stopping_rounds=5, 
              eval_set=[(X_valid, y_valid)], 
              verbose=False)
  preds_xgb = model_xgb.predict(X_valid)
  print(mean_squared_error(y_valid, preds_xgb, squared=False))

  return model_xgb

In [ ]:
models = []

for est in [1000, 2000, 3000, 4000, 5000]:
  model = train_regressor(est, 0.05)
  models.append(model)
